# 향수 상세정보 크롤링 데이터 적재 스크립트
## 관련 테이블 정보
- evaluation_code, evaluation_field_option, fragrantica_evaluation
- perfume, perfume_brand
- pefume_accord, note, perfume_note

## TODO
- 이슈 사항에 대한 처리
- perfume_brand: 초기 display_order, image_url 추가
- perfume: image_url에 대해 s3 url로 변경(현재 프라그란티카 url)
- 한영 dictionary를 이용하여 한글로 변환 후 적재하는 로직 추가

In [1]:
!pip install pymysql

In [1]:
# test
import pymysql


# MySQL 연결 설정
conn = pymysql.connect(
    host='127.0.0.1', 
    port=3306,
    user='root', 
    password='secret', 
    db='purple', 
    charset='utf8'
)

cursor = conn.cursor()

sql = "SHOW TABLES;"
cursor.execute(sql)
result = cursor.fetchall()
for table in result:
    print(table[0])

conn.commit()
conn.close()

accord
brand
complaint
evaluation_statistic
flyway_schema_history
fragrantica_evaluation
likes
mood
note
perfume
perfume_accord
review
review_evaluation
review_mood
star_rating
star_rating_statistic
user
user_accord


In [2]:
import os
import json

# JSON 데이터
DIRECTORY_PATH = './crawling_data'
JSON_DATA_LIST = []
for filename in os.listdir(DIRECTORY_PATH):
    if filename.endswith('.json'):
        file_path = os.path.join(DIRECTORY_PATH, filename)
        
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
            JSON_DATA_LIST.append(data)

In [3]:
# code set
evaluation_codes = {
	"LONGEVITY": {
		"code": "EF001",
		"name": "지속력",
		"type": "FIELD",
		"options": {
			"very weak": {
				"code": "EO101",
				"name": "매우 약함",
				"type": "OPTION"
			},
			"weak": {
				"code": "EO102",
				"name": "약함",
				"type": "OPTION"
			},
			"moderate": {
				"code": "EO103",
				"name": "보통",
				"type": "OPTION"
			},
			"long lasting": {
				"code": "EO104",
				"name": "오래감",
				"type": "OPTION"
			},
			"eternal": {
				"code": "EO105",
				"name": "매우 오래감",
				"type": "OPTION"
			}
		}
	},
	"SILLAGE": {
		"code": "EF002",
		"name": "시야주",
		"type": "FIELD",
		"options": {
			"intimate": {
				"code": "EO201",
				"name": "향 여운이 약함",
				"type": "OPTION"
			},
			"moderate": {
				"code": "EO202",
				"name": "보통",
				"type": "OPTION"
			},
			"strong": {
				"code": "EO203",
				"name": "향 여운이 강함",
				"type": "OPTION"
			},
			"enormous": {
				"code": "EO204",
				"name": "향 여운이 매우 강함",
				"type": "OPTION"
			}
		}
	},
	"seasonData": {
		"code": "EF003",
		"name": "계절감/시간",
		"type": "FIELD",
		"options": {
			"spring": {
				"code": "EO301",
				"name": "봄",
				"type": "OPTION"
			},
			"summer": {
				"code": "EO302",
				"name": "여름",
				"type": "OPTION"
			},
			"fall": {
				"code": "EO303",
				"name": "가을",
				"type": "OPTION"
			},
			"winter": {
				"code": "EO304",
				"name": "겨울",
				"type": "OPTION"
			},
			"day": {
				"code": "EO305",
				"name": "낮",
				"type": "OPTION"
			},
			"night": {
				"code": "EO306",
				"name": "밤",
				"type": "OPTION"
			}
		}
	},
	"GENDER": {
		"code": "EF004",
		"name": "성별",
		"type": "FIELD",
		"options": {
			"male": {
				"code": "EO401",
				"name": "남성",
				"type": "OPTION"
			},
			"more male": {
				"code": "EO402",
				"name": "남성에 가까운",
				"type": "OPTION"
			},
			"unisex": {
				"code": "EO403",
				"name": "중성",
				"type": "OPTION"
			},
			"more female": {
				"code": "EO404",
				"name": "여성에 가까운",
				"type": "OPTION"
			},
			"female": {
				"code": "EO405",
				"name": "여성",
				"type": "OPTION"
			}
		}
	}
}


# moods
moods = [
    "귀여운",
    "관능적인",
    "청순한",
    "우아한",
    "섹시한",
    "스포티한",
    "중후한",
    "마초적인",
    "케주얼한",
    "편안한",
    "깔끔한",
    "세련된"
]

# brands
brands = {
	"Chanel": { "brand_name": "샤넬", "display_order": 1 },
	"Jo Malone London": { "brand_name": "조말론", "display_order": 2 },
	"Diptyque": { "brand_name": "딥디크", "display_order": 3 },
	"Dior": { "brand_name": "크리스찬디올", "display_order": 4 },
	"Tom Ford": { "brand_name": "톰포드", "display_order": 5 },
	"Bvlgari": { "brand_name": "불가리", "display_order": 6 },
	"Byredo": { "brand_name": "바이레도", "display_order": 7 },
	"Creed": { "brand_name": "크리드", "display_order": 8 },
	"Lanvin": { "brand_name": "랑방", "display_order": 9 },
	"Burberry": { "brand_name": "버버리", "display_order": 10 },
	"Gucci": { "brand_name": "구찌", "display_order": 11 },
	"Le Labo": { "brand_name": "르라보", "display_order": 12 },
	"Chloé": { "brand_name": "끌로에", "display_order": 13 },
	"Versace": { "brand_name": "베르사체", "display_order": 14 },
	"Lush": { "brand_name": "러쉬", "display_order": 15 },
	"Montblanc": { "brand_name": "몽블랑", "display_order": 16 },
	"Jimmy Choo": { "brand_name": "지미추", "display_order": 17 },
	"Clean": { "brand_name": "클린", "display_order": 18 },
	"Kenzo": { "brand_name": "겐조", "display_order": 19 },
	"Marc Jacobs": { "brand_name": "마크제이콥스", "display_order": 20 },
	"Acqua di Parma": { "brand_name": "아쿠아디파르마", "display_order": 21 },
	"John Varvatos": { "brand_name": "존바바토스", "display_order": 22 },
	"Ferrari": { "brand_name": "페라리", "display_order": 23 },
	"Kiehl's": { "brand_name": "키엘", "display_order": 24 },
	"Lolita Lempicka": { "brand_name": "롤리타렘피카", "display_order": 25 },
	"Calvin Klein": { "brand_name": "캘빈클라인", "display_order": 26 },
	"Penhaligon's": { "brand_name": "펜할리곤스", "display_order": 27 },
	"Salvatore Ferragamo": {
		"brand_name": "살바토레페라가모",
		"display_order": 28
	},
	"Giorgio Armani": { "brand_name": "조르지오아르마니", "display_order": 29 }
}

In [4]:
from datetime import datetime

def generate_id():
    '''
    현재 시간을 yyyyMMddHHmmssSSSSS 형식으로 포맷팅 후 int로 변환
    '''
    return int(datetime.now().strftime('%Y%m%d%H%M%S%f')[:-1])

def get_or_generate_id(query, variables):
    cursor.execute(query, variables)
    result = cursor.fetchone()
    return generate_id() if result == None else result[0]

In [5]:
# sql_queries

# insert
insert_brand_query = (
    "INSERT IGNORE INTO "
    "brand(brand_name, image_url, display_order) "
    "VALUES(%s, '', %s);"
)

insert_perfume_query = (
    "INSERT INTO "
    "perfume("
    "perfume_id, brand_name, perfume_name, image_url"
    ") "
    "VALUES(%s, %s, %s, %s) "
    "ON DUPLICATE KEY UPDATE "
    "image_url = VALUES(image_url);"
)

insert_fragrantica_evaluation_query = (
    "INSERT INTO "
    "fragrantica_evaluation("
    "perfume_id, field_code, option_code, votes"
    ") "
    "VALUES(%s, %s, %s, %s) "
    "ON DUPLICATE KEY UPDATE "
    "votes = VALUES(votes);"
)

insert_note_query = (
    "INSERT IGNORE INTO "
    "note(perfume_id, note_name, note_type) "
    "VALUES(%s, %s, %s);"
)

insert_mood_query = (
    "INSERT IGNORE INTO "
    "mood(mood_name) "
    "VALUES(%s);"
)

insert_accord_query = (
    "INSERT IGNORE INTO "
    "accord(accord_name) "
    "VALUES(%s);"
)

insert_perfume_accord_query = (
    "INSERT INTO "
    "perfume_accord("
    "perfume_id, accord_name, value"
    ") "
    "VALUES(%s, %s, %s) "
    "ON DUPLICATE KEY UPDATE "
    "value = VALUES(value);"
)

# select id
select_perfume_perfume_id_query = (
    "SELECT perfume_id "
    "FROM perfume "
    "WHERE brand_name=%s AND perfume_name=%s;"
)

In [6]:
import pymysql
            
# MySQL 연결 설정
conn = pymysql.connect(
    host='127.0.0.1', 
    port=3306,
    user='root', 
    password='secret', 
    db='purple', 
    charset='utf8'
)

cursor = conn.cursor()

# TODO: mood 추가
print("===========start inserting default setting...===========")
print("mood>>> start...")
for mood_name in moods:
    cursor.execute(
            insert_mood_query, 
            (mood_name)
        )
print("mood>>> ...end.")


print("===========start inserting crawl data...===========")

field_key_list = ["seasonData", "LONGEVITY", "SILLAGE", "GENDER"]

for brand_data in JSON_DATA_LIST:
    for perfume_data in brand_data:
        # perfume_brand
        # TODO: 초기 display_order, image_url 추가
        brand = perfume_data['companyName']
        brand_name = brands[brand]["brand_name"]
        display_order = brands[brand]["display_order"]
        cursor.execute(insert_brand_query, (brand_name, display_order))
        
        # perfume 
        # TODO: image_url에 대해 s3 url로 변경(현재 프라그란티카 url)
        perfume_name = perfume_data['perfumeName']
        print(f"start: {brand_name} > {perfume_name}")
        
        perfume_id = get_or_generate_id(
            select_perfume_perfume_id_query,
            (brand_name, perfume_name)
        )
        image_url = perfume_data.get('thumbnailSrc')
        cursor.execute(
            insert_perfume_query, 
            (perfume_id, brand_name, perfume_name, image_url)
        )
        
        # fragrantica_evaluation(seasonData)
        field_code = evaluation_codes['seasonData']['code']
        for option_key, votes in perfume_data['seasonData'].items():
            option_code = evaluation_codes['seasonData']['options'][option_key]['code']
            cursor.execute(
                insert_fragrantica_evaluation_query, 
                (perfume_id, field_code, option_code, votes)
            )
        
        # fragrantica_evaluation(perfumeProperties)
        for field_key, field_data in perfume_data['perfumeProperties'].items():
            if (field_key not in field_key_list): continue;
            field_code = evaluation_codes[field_key]['code']
            
            for option_key, votes in field_data.items():
                option_code = evaluation_codes[field_key]['options'][option_key]['code']
                cursor.execute(
                    insert_fragrantica_evaluation_query, 
                    (perfume_id, field_code, option_code, votes)
                )
        
        # accord & perfume_accord
        for accord_name, accord_value in perfume_data['notes'].items():
            cursor.execute(
                insert_accord_query,
                (accord_name)
            )
            cursor.execute(
                insert_perfume_accord_query, 
                (perfume_id, accord_name, accord_value)
            )
        
        
        if (perfume_data['noteTopMiddleBase'].get("Top Notes")):
            for note_name in perfume_data['noteTopMiddleBase'].get("Top Notes"):
                # note
                note_type = "TOP"
                cursor.execute(
                    insert_note_query,
                    (perfume_id, note_name, note_type)
                )
                
                
                
        if (perfume_data['noteTopMiddleBase'].get("Middle Notes")):
            for note_name in perfume_data['noteTopMiddleBase'].get("Middle Notes"):
                # note
                note_type = "MIDDLE"
                cursor.execute(
                    insert_note_query,
                    (perfume_id, note_name, note_type)
                )
                
        
        if (perfume_data['noteTopMiddleBase'].get("Base Notes")):
            for note_name in perfume_data['noteTopMiddleBase'].get("Base Notes"):
                # note
                note_type = "BASE"
                cursor.execute(
                    insert_note_query,
                    (perfume_id, note_name, note_type)
                )
        print(f"no error: {brand_name} > {perfume_name}")
        
        
conn.commit()
print("===========perfume data committed.===========")

cursor.close()
conn.close()


===========start inserting default setting...===========
mood>>> start...
mood>>> ...end.
===========start inserting crawl data...===========
start: 마크제이콥스 > Daisy Love
no error: 마크제이콥스 > Daisy Love
start: 마크제이콥스 > Daisy Eau So Fresh
no error: 마크제이콥스 > Daisy Eau So Fresh
start: 마크제이콥스 > Daisy Dream
no error: 마크제이콥스 > Daisy Dream
start: 마크제이콥스 > Perfect
no error: 마크제이콥스 > Perfect
start: 마크제이콥스 > Daisy
no error: 마크제이콥스 > Daisy
start: 마크제이콥스 > Daisy Love Pop
no error: 마크제이콥스 > Daisy Love Pop
start: 마크제이콥스 > Daisy Love Eau So Sweet Petals
no error: 마크제이콥스 > Daisy Love Eau So Sweet Petals
start: 마크제이콥스 > Daisy Love Skies
no error: 마크제이콥스 > Daisy Love Skies
start: 마크제이콥스 > Daisy Ever So Fresh
no error: 마크제이콥스 > Daisy Ever So Fresh
start: 마크제이콥스 > Perfect Intense
no error: 마크제이콥스 > Perfect Intense
start: 바이레도 > Blanche
no error: 바이레도 > Blanche
start: 바이레도 > La Tulipe
no error: 바이레도 > La Tulipe
start: 바이레도 > Rose Of No Man's Land
no error: 바이레도 > Rose Of No Man's Land
start: 바이레도 > Casablanca 

no error: 러쉬 > Dirty
start: 러쉬 > Rose Jam 2019
no error: 러쉬 > Rose Jam 2019
start: 러쉬 > Vanillary
no error: 러쉬 > Vanillary
start: 러쉬 > Karma
no error: 러쉬 > Karma
start: 러쉬 > Breath Of God
no error: 러쉬 > Breath Of God
start: 러쉬 > All Good Things
no error: 러쉬 > All Good Things
start: 러쉬 > Sun
no error: 러쉬 > Sun
start: 러쉬 > Imogen Rose
no error: 러쉬 > Imogen Rose
start: 러쉬 > Devil's Night Cap
no error: 러쉬 > Devil's Night Cap
start: 러쉬 > Amelie Mae
no error: 러쉬 > Amelie Mae
start: 샤넬 > Chanel No 5 Eau de Parfum
no error: 샤넬 > Chanel No 5 Eau de Parfum
start: 샤넬 > Coco Mademoiselle
no error: 샤넬 > Coco Mademoiselle
start: 샤넬 > Bleu de Chanel Eau de Parfum
no error: 샤넬 > Bleu de Chanel Eau de Parfum
start: 샤넬 > Chance Eau Tendre
no error: 샤넬 > Chance Eau Tendre
start: 샤넬 > Coco Mademoiselle Intense
no error: 샤넬 > Coco Mademoiselle Intense
start: 샤넬 > Allure Homme Sport
no error: 샤넬 > Allure Homme Sport
start: 샤넬 > Chance Eau Tendre Eau de Parfum
no error: 샤넬 > Chance Eau Tendre Eau de Parfum
s

no error: 톰포드 > Rose Prick
start: 톰포드 > Noir de Noir
no error: 톰포드 > Noir de Noir
start: 톰포드 > Bitter Peach
no error: 톰포드 > Bitter Peach
start: 톰포드 > Soleil Neige
no error: 톰포드 > Soleil Neige
start: 톰포드 > Electric Cherry
no error: 톰포드 > Electric Cherry
start: 톰포드 > Myrrhe Mystère
no error: 톰포드 > Myrrhe Mystère
start: 구찌 > Flora Gorgeous Gardenia Eau de Parfum
no error: 구찌 > Flora Gorgeous Gardenia Eau de Parfum
start: 구찌 > Flora Gorgeous Jasmine
no error: 구찌 > Flora Gorgeous Jasmine
start: 구찌 > A Song For The Rose Eau de Parfum
no error: 구찌 > A Song For The Rose Eau de Parfum
start: 구찌 > Guilty Pour Homme Eau de Parfum
no error: 구찌 > Guilty Pour Homme Eau de Parfum
start: 구찌 > Gucci Guilty Absolute pour Femme
no error: 구찌 > Gucci Guilty Absolute pour Femme
start: 구찌 > Gucci Bloom Intense
no error: 구찌 > Gucci Bloom Intense
start: 구찌 > Gucci Guilty Love Edition Pour Homme
no error: 구찌 > Gucci Guilty Love Edition Pour Homme
start: 구찌 > Gucci Guilty Eau de Parfum Intense Pour Femme
no erro

# 이슈: 필요 데이터가 없는 경우 확인

In [8]:
total_perfume_count = 0
note_error_perfume_count = 0
image_not_exist_perfume_count = 0

print("===========start checking===========")
for brand_data in JSON_DATA_LIST:
    total_perfume_count += len(brand_data)
    for perfume_data in brand_data:
        # perfume_brand
        brand_name = perfume_data['companyName']
        
        # perfume 
        perfume_name = perfume_data['perfumeName']
        
        image_url = perfume_data.get('thumbnailSrc')
        if (image_url == None):
            image_not_exist_perfume_count += 1
            print(f"향수 섬네일 없음 <- 브랜드명: {brand_name}, 향수명: {perfume_name}")
        
        
        top_notes = perfume_data['noteTopMiddleBase'].get("Top Notes")
        middle_notes = perfume_data['noteTopMiddleBase'].get("Middle Notes")
        base_notes = perfume_data['noteTopMiddleBase'].get("Base Notes")
        if (top_notes == middle_notes == base_notes == None):
            note_error_perfume_count += 1
            print(f"Top/Middle/Base 노트 정보 없음 <- 브랜드명: {brand_name}, 향수명: {perfume_name}")
            
        elif (top_notes == None):
            print(f"Top 노트 정보 없음 <- 브랜드명: {brand_name}, 향수명: {perfume_name}")
                
                
                
        elif (middle_notes == None):
            print(f"Middle 노트 정보 없음 <- 브랜드명: {brand_name}, 향수명: {perfume_name}")
                
        
        elif (base_notes == None):
            print(f"Base 노트 정보 없음 <- 브랜드명: {brand_name}, 향수명: {perfume_name}")
        
        
        
print("===========all checked===========")
print(f"총 향수 개수: {total_perfume_count}")
print(f"이미지 없는 향수 개수: {image_not_exist_perfume_count}")
print(f"Top/Middle/Base 노트 정보 없는 향수 개수: {note_error_perfume_count}")

===========start checking===========
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Penhaligon's, 향수명: Blenheim Bouquet
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Penhaligon's, 향수명: Clandestine Clara
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Penhaligon's, 향수명: Much Ado About The Duke
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Jo Malone London, 향수명: Poppy & Barley
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Jo Malone London, 향수명: Cypress & Grapevine Cologne Intense
향수 섬네일 없음 <- 브랜드명: Jo Malone London, 향수명: Ginger Beer Cologne
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Acqua di Parma, 향수명: Acqua di Parma Blu Mediterraneo - Fico di Amalfi
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Acqua di Parma, 향수명: Essenza di Colonia
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Acqua di Parma, 향수명: Peonia Nobile
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Chloé, 향수명: Jasminum Sambac
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Le Labo, 향수명: Bergamote 22
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Le Labo, 향수명: Santal 33
Top/Middle/Base 노트 정보 없음 <- 브랜드명: Le Labo, 향수명: The Matcha 26
Top/Middle/Base 노트 정보 없음 <- 